<a href="https://colab.research.google.com/github/Gayeon6423/BusinessAI-Capston/blob/main/3_Data_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

Mounted at /content/drive


# Data

In [ ]:
kospi = pd.read_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Preprocessing_Data/Preprocessing_Neumeric_Data/neumeric_preprocess_kospi.csv')
kosdaq = pd.read_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Preprocessing_Data/Preprocessing_Neumeric_Data/neumeric_preprocess_kosdaq.csv')

kospi_research = pd.read_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Stock Index Data/kospi_research.csv')
kosdaq_research = pd.read_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Stock Index Data/kosdaq_research.csv')

Sentiment_Kospi_KOFINBERT = pd.read_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Sentiment_Score_Data/Sentiment_Score_all_ZSCORE_Kospi_KOFINBERT.csv')
Sentiment_Kospi_KLUEBERT = pd.read_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Sentiment_Score_Data/Sentiment_Score_all_ZSCORE_Kospi_KLUEBERT.csv')
Sentiment_Kosdaq_KOFINBERT = pd.read_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Sentiment_Score_Data/Sentiment_Score_all_ZSCORE_Kosdaq_KOFINBERT.csv')
Sentiment_Kosdaq_KLUEBERT = pd.read_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Sentiment_Score_Data/Sentiment_Score_all_ZSCORE_Kosdaq_KLUEBERT.csv')

In [ ]:
Sentiment_Kospi_KOFINBERT = Sentiment_Kospi_KOFINBERT.drop(columns = ['kofinbert_score_preprocess_context_kospi','kofinbert_score_title_plus_preprocess_content_kospi'])
Sentiment_Kospi_KLUEBERT = Sentiment_Kospi_KLUEBERT.drop(columns = ['klue_bert_score_preprocess_context','klue_bert_score_title_plus_preprocess_context','klue_bert_score_title_plus_summarize_content'])
Sentiment_Kosdaq_KOFINBERT = Sentiment_Kosdaq_KOFINBERT.drop(columns = ['kofinbert_score_preprocess_context_kosdaq','kofinbert_score_title_plus_preprocess_content_kosdaq'])
Sentiment_Kosdaq_KLUEBERT = Sentiment_Kosdaq_KLUEBERT.drop(columns = ['klue_bert_score_preprocess_context','klue_bert_score_title_plus_preprocess_context','klue_bert_score_title_plus_summarize_content'])

In [ ]:
# # 필요없는 칼럼 제거
# kospi = kospi.drop(['dji_open','dji_high','dji_low','us500_open','us500_high','us500_low','ex_AM_open','ex_AM_high','ex_AM_low',
#                     'ex_JP_open','ex_JP_high','ex_JP_low','gold_open','gold_high','gold_low','oil_open','oil_high','oil_low'],axis=1)
# kosdaq = kosdaq.drop(['dji_open','dji_high','dji_low','us500_open','us500_high','us500_low','ex_AM_open','ex_AM_high','ex_AM_low',
#                     'ex_JP_open','ex_JP_high','ex_JP_low','gold_open','gold_high','gold_low','oil_open','oil_high','oil_low'],axis=1)

In [ ]:
# Date 칼럼 전처리
Sentiment_Kospi_KLUEBERT['prediction'] = pd.to_datetime(Sentiment_Kospi_KLUEBERT['prediction'], format='%Y-%m-%d')
Sentiment_Kospi_KLUEBERT['prediction'] = Sentiment_Kospi_KLUEBERT['prediction'].dt.strftime('%Y%m%d')
Sentiment_Kospi_KLUEBERT = Sentiment_Kospi_KLUEBERT.rename(columns = {'prediction':'Date'})
Sentiment_Kospi_KLUEBERT['Date'] = Sentiment_Kospi_KLUEBERT['Date'].astype('int64')

Sentiment_Kosdaq_KLUEBERT['prediction'] = pd.to_datetime(Sentiment_Kosdaq_KLUEBERT['prediction'], format='%Y-%m-%d')
Sentiment_Kosdaq_KLUEBERT['prediction'] = Sentiment_Kosdaq_KLUEBERT['prediction'].dt.strftime('%Y%m%d')
Sentiment_Kosdaq_KLUEBERT = Sentiment_Kosdaq_KLUEBERT.rename(columns = {'prediction':'Date'})
Sentiment_Kosdaq_KLUEBERT['Date'] = Sentiment_Kosdaq_KLUEBERT['Date'].astype('int64')

# 중복행 제거
Sentiment_Kospi_KOFINBERT = Sentiment_Kospi_KOFINBERT.drop_duplicates(subset = ['Date'])
Sentiment_Kosdaq_KOFINBERT = Sentiment_Kosdaq_KOFINBERT.drop_duplicates(subset = ['Date'])

In [ ]:
# MinMax Scalaer 적용
scaler = MinMaxScaler()
kospi_research[['research_kospi']] = scaler.fit_transform(kospi_research[['research_kospi']])
kosdaq_research[['research_kosdaq']] = scaler.fit_transform(kosdaq_research[['research_kosdaq']])

In [ ]:
# 주가 데이터 및 감정 스코어 병합(주가 데이터의 날짜 기준)
kospi_all_kofinbert = pd.merge(kospi, Sentiment_Kospi_KOFINBERT, on = 'Date', how = 'left')
kospi_all_kluebert = pd.merge(kospi, Sentiment_Kospi_KLUEBERT, on = 'Date', how = 'left')
kosdaq_all_kofinbert = pd.merge(kospi, Sentiment_Kosdaq_KOFINBERT, on = 'Date', how = 'left')
kosdaq_all_kluebert = pd.merge(kospi, Sentiment_Kosdaq_KLUEBERT, on = 'Date', how = 'left')

# 감정 데이터 병합
kospi_sen = pd.concat([kospi_all_kofinbert, kospi_all_kluebert.iloc[:,37:39]], axis=1)
kosdaq_sen = pd.concat([kosdaq_all_kofinbert, kosdaq_all_kluebert.iloc[:,37:39]], axis=1)

In [ ]:
# research 데이터 병합
kospi_sen_all = pd.merge(kospi_sen, kospi_research, on = 'Date', how = 'left')
col = kospi_sen_all['research_kospi']
kospi_sen_all = kospi_sen_all.drop(columns = 'research_kospi')
kospi_sen_all.insert(1,'research_kospi', col)

kosdaq_sen_all = pd.merge(kosdaq_sen, kosdaq_research, on = 'Date', how = 'left')
col = kosdaq_sen_all['research_kosdaq']
kosdaq_sen_all = kosdaq_sen_all.drop(columns = 'research_kosdaq')
kosdaq_sen_all.insert(1,'research_kosdaq', col)

# 소수점 3자리 수
kospi_sen_all = kospi_sen_all.round(3)
kosdaq_sen_all = kosdaq_sen_all.round(3)

In [ ]:
kospi_sen_all = kospi_sen_all.dropna()
kosdaq_sen_all = kosdaq_sen_all.dropna()

In [ ]:
# 데이터 저장
kospi_sen_all.to_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Total_Data/kospi_sen_all.csv', encoding='utf=-8-sig', index=False)
kosdaq_sen_all.to_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Total_Data/kosdaq_sen_all.csv', encoding='utf=-8-sig', index=False)

# Kospi data 하루씩 늦게 반영하기

In [ ]:
# 데이터 불러오기
kospi_df = pd.read_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Total_Data/kospi_sen_all.csv')

In [ ]:
# Kospi_close 변수 제외한 Kospi 문자열이 포함된 데이터 하루씩 뒤로 밀기
def shift_lows(column):
  kospi_df[column] = kospi_df[column].shift(1)

col = [i for i in kospi_df.columns if "Kospi" in i and i != 'Kospi_close']
for i in col:
  shift_lows(i)

In [ ]:
# 새로운 데이터셋으로 저장
df = kospi_df.dropna().reset_index(drop=True)
df.to_csv("/content/drive/MyDrive/산업 AI 캡스톤/DATA/Total_Data/shift_kospi_sen_all.csv", index=False)